In [ ]:
from langchain import hub
from langchain_openai import ChatOpenAI

prompt = hub.pull('bsorrentino/image_to_plantuml')

print( prompt.format() )


In [ ]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.schema.messages import HumanMessage


def converImageToDiagram(image_url_or_data: str):

    prompt2 = hub.pull('bsorrentino/image_to_plantuml').format()
    prompt1 = """
    extract from diagrams represented in the image the following information:

    - Diagram tipology - ["<diagram type>"] Eg. ["sequence diagram"], ["class diagram"], ["state machine diagram"], etc.
    - each component and its shape - ["<component name>":"<shape>"]. Eg. : ["Action": "rectangle"], [ "Process": "circle"] , ["Step": "oval"], ["User": "stick"],  etc.
    - each relation and its name - ["<source component>", "<relation name>", "<target component>"]. Eg. ["Element", "depends on", "Process", ], ["Process", "has", "Step" ], etc.
   
    Assume that:
    - Shape must be one of the following: rectangle, circle, stick, cloud, database, actor, cylinder, rhombus
    - If component cointains other components, its shape must be container
    - lines and arrows respresent the relations between components and their name is the label describing the relation itself
    - Relation must non appear in component list and viceversa  
    """
    prompt = """
    extract from diagrams represented in the image the following information:

    - Diagram tipology - ["<diagram type>"]. Eg. ["sequence diagram"], ["class diagram"], ["state machine diagram"], etc.
    - each component and its shape - ["<component name>":"<shape>"]. Eg. ["Action": "rectangle"], [ "Process": "circle"] , ["Step": "oval"], ["User": "stick"],  etc.
    - each relation and its name - ["<source component>", "<relation name>", "<target component>"]. Eg. ["Element", "depends on", "Process", ], ["Process", "has", "Step" ], etc.
   
    Assume that:
    - lines and arrows respresent the relations between components and their name is the label describing the relation itself
    - relations must non appear in components list and viceversa  
    - If component cointains other components, its shape must be "container" and must contains the components list
    """

    prompt_diagram = """
    describe the diagram in the image step by step so we can translate it into diagram-as-code syntax.

    start description with:
    - Diagram tipology: ["<diagram type>"]. Eg. ["sequence diagram"], ["class diagram"], ["state machine diagram"], etc.
    - Diagram summary (max one line) or tile: ["<diagram title|summary>"] 

    """

    openai = ChatOpenAI(model="gpt-4-vision-preview",
        max_tokens=2000,
        temperature=0.5
    )

    response = openai.invoke([
        HumanMessage(content=[
                {
                    "type": "text", 
                    "text": prompt_diagram
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": image_url_or_data
                    },
                },
            ]
        )
    ])

    return response.content



In [ ]:
import gradio as gr
import base64, io
import PIL
from os import path 

def process_image(image: PIL.Image.Image):

    try:
        in_mem_file = io.BytesIO()
    
        image.save( in_mem_file, format = image.format if image.format is not None else 'PNG' )
        encoded_string = base64.b64encode(in_mem_file.getvalue())
        
        imageData = "data:image/png;base64," + encoded_string.decode('utf-8')

        result =  converImageToDiagram(imageData) 

        print( result )

        return result

    except Exception as e:
        return "Error: " + str(e)


demo = gr.Interface(
    fn=process_image,
    inputs=[
        gr.Image(type="pil")
    ],
    outputs=[
        gr.Markdown()
    ],
    # flagging_options=["blurry", "incorrect", "other"],
    examples=[
        path.join( "..", "..", "assets", "img_p3_1.png" ),
        path.join( "..", "..", "assets", "architecture2.png" ),
        path.join( "..", "..", "assets", "http-streaming.png" ),
    ],
)

# if __name__ == "__main__":
#     demo.launch()

demo.launch()